In [1]:
with open("raw_logs/files.txt", "r") as file:
    filenames = file.readlines()

In [2]:
filenames = list(map(str.strip, filenames))
filenames = list(filter(lambda x: x[0] == 'D', filenames))

In [3]:
import pandas as pd

files = pd.DataFrame(columns=["name", "time", "num_records", "min_time_offset", "max_time_offset"])
records_columns = [
    "time",
    "type",
    "id",
    "dlc",
    "data0",
    "data1",
    "data2",
    "data3",
    "data4",
    "data5",
    "data6",
    "data7",
    "filename"
]
records = pd.DataFrame(columns=records_columns)
files_list = list()
records_dfs = list()

In [4]:

from datetime import datetime, timedelta


def extractDateInfo(rawtime):
    days = timedelta(days=rawtime)
    date = datetime(year=1899, month=12, day=30) + days
    return pd.to_datetime(pd.Series({
        "year": date.year,
        "month": date.month,
        "day": date.day,
        "hour": date.hour,
        "minute": date.minute,
        "second": date.second,
        "microsecond": date.microsecond,

    }))


In [5]:
from tqdm.notebook import tqdm

for filename in tqdm(filenames):
    with open(f"raw_logs/{filename}", "r") as file:
        lines = file.readlines()
    startTime = float(lines[1][12:])
    records = []
    for line in lines[14:]:
        records.append(list(filter(lambda x: x != "" and x != "\n", line.split(" ")))[1:] + [filename])
    tmp = pd.DataFrame.from_records(records, columns=records_columns)
    tmp["rawtime"] = (tmp["time"].astype(float) / (1000 * 60 * 60 * 24)) + startTime
    dates = pd.DataFrame.from_records(tmp["rawtime"].map(extractDateInfo))
    tmp = pd.merge(tmp, dates, left_index=True, right_index=True)
    records_dfs.append(tmp)
    files_list.append([filename, startTime, len(records), tmp["time"].min(), tmp["time"].max()])
records = pd.concat(records_dfs)

  0%|          | 0/39 [00:00<?, ?it/s]

In [6]:
records.to_csv("records.csv")

In [7]:
len(records)

3502594